In [1]:
import pandas as pd

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
tp_df = pd.read_csv("dataset/train_true_positive_features.csv")
tn_df = pd.read_csv("dataset/train_true_negative_features.csv")
df = pd.concat([tp_df, tn_df])
df = df.reset_index(drop = True)

In [4]:
df = df[['vgg_cosine', 'vgg_euclidean_l2'
         , 'facenet_cosine', 'facenet_euclidean_l2'
         , 'openface_cosine', 'openface_euclidean_l2'
         , 'is_related']]

In [5]:
df.sample(5)

,vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
74719,0.560566,1.058836,0.886210,1.331322,0.586833,1.083359,1
303703,0.668578,1.156354,0.997718,1.412599,0.957214,1.383628,0
314107,0.590811,1.087025,1.166196,1.527217,1.244372,1.577575,0
320077,0.574306,1.071733,0.694420,1.178490,0.354096,0.841542,0
303855,0.585434,1.082067,1.149306,1.516117,0.575240,1.072604,0


# Train Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test = train_test_split(df, test_size=0.15, random_state=17)

# H2O AutoML

In [8]:
import h2o
from h2o.automl import H2OAutoML

In [9]:
h2o.__version__

'3.26.0.3'

In [10]:
# specify maximum memory size and number of threads based on your system

In [11]:
import multiprocessing
multiprocessing.cpu_count()

72

In [12]:
import psutil
psutil.virtual_memory()

svmem(total=1622764929024, available=680730955776, percent=58.1, used=938275319808, free=587081805824, active=921876504576, inactive=94499237888, buffers=0, cached=97407803392, shared=876011520, slab=2707632128)

In [13]:
#h2o.init()
h2o.init(ip="127.0.0.1", max_mem_size_GB = 200, nthreads = 10)

Checking whether there is an H2O instance running at http://127.0.0.1:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_212"; OpenJDK Runtime Environment (build 1.8.0_212-b04); OpenJDK 64-Bit Server VM (build 25.212-b04, mixed mode)
  Starting server from /workspaces/sefik/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpsbq_2v_e
  JVM stdout: /tmp/tmpsbq_2v_e/h2o_sefik_started_from_python.out
  JVM stderr: /tmp/tmpsbq_2v_e/h2o_sefik_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Istanbul
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_sefik_5x06kk
H2O cluster total nodes:,1
H2O cluster free memory:,177.8 Gb
H2O cluster total cores:,72
H2O cluster allowed cores:,10
H2O cluster status:,"accepting new members, healthy"


In [14]:
#pandas to h2o frame
hf = h2o.H2OFrame(x_train)

#alternatively, we can directly import h2o frame instead of converting from pandas
#hf = h2o.import_file('dataset/x_train.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
hf.head()

vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
0.461026,0.960236,0.759346,1.23235,0.380667,0.872545,1
0.523298,1.02303,0.820089,1.28069,0.572227,1.06979,1
0.57931,1.07639,0.834568,1.29195,0.411612,0.907317,1
0.661336,1.15007,1.25314,1.58312,0.809752,1.2726,0
0.380073,0.871864,0.419412,0.915873,0.386296,0.878972,1
0.586928,1.08345,0.980849,1.40061,0.791823,1.25843,0
0.543978,1.04305,0.900194,1.34179,0.88473,1.33021,1
0.378311,0.86984,0.799367,1.26441,0.467552,0.967008,0
0.483638,0.983502,0.590177,1.08644,0.411636,0.907343,1
0.874395,1.32242,0.935428,1.36779,1.11803,1.49535,0


In [16]:
y_label = "is_related"
x_labels = list(df.drop(columns=[y_label]).columns)

In [17]:
x_labels

['vgg_cosine',
 'vgg_euclidean_l2',
 'facenet_cosine',
 'facenet_euclidean_l2',
 'openface_cosine',
 'openface_euclidean_l2']

In [18]:
#this is a binary classification problem. convert is_related column to enum type instead of numerical
#otherwise, it would be a regression problem
hf[y_label] = hf[y_label].asfactor()

In [ ]:
#hf.describe()

In [21]:
aml = H2OAutoML(max_runtime_secs=60*60*2)

In [22]:
aml.train(x = x_labels, y = y_label, training_frame = hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%


# Leaderboard

In [23]:
lb = aml.leaderboard

In [26]:
#lb.head()
lb.head(rows=lb.nrows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_3_AutoML_20190903_230729,0.724614,0.581255,0.343654,0.445343,0.19833
StackedEnsemble_AllModels_AutoML_20190903_230729,0.724614,0.582056,0.34564,0.445599,0.198559
StackedEnsemble_BestOfFamily_AutoML_20190903_230729,0.724574,0.582107,0.342329,0.445619,0.198577
XGBoost_1_AutoML_20190903_230729,0.722517,0.58275,0.342604,0.446046,0.198957
GLM_grid_1_AutoML_20190903_230729_model_1,0.718098,0.620438,0.348661,0.463758,0.215071
XGBoost_2_AutoML_20190903_230729,0.71745,0.586581,0.350728,0.447797,0.200522


In [ ]:
#saved_model = h2o.save_model(aml.leader, path = "")

In [ ]:
#saved_model

# Validation Score

In [27]:
hf_val = h2o.H2OFrame(x_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
predictions = aml.predict(hf_val)
predictions_pd = predictions.as_data_frame()

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [29]:
predictions_pd.head()

,predict,p0,p1
0,1,0.299512,0.700488
1,0,0.820793,0.179207
2,1,0.542932,0.457068
3,1,0.490345,0.509655
4,1,0.647635,0.352365


In [31]:
actuals = hf_val['is_related']
actuals_pd = actuals.as_data_frame()

In [34]:
classified = 0
for i in range(0, predictions_pd.shape[0]):
    prediction = predictions_pd.iloc[i].values[0]
    #prediction = predictions_pd.iloc[i].values[1] #probability of y = 0
    #prediction = predictions_pd.iloc[i].values[2] #probability of y = 1
    
    actual = actuals_pd.iloc[i].values[0]
    
    if actual == prediction:
        classified = classified + 1

In [35]:
print("accuracy on test set is: ",100*classified / predictions_pd.shape[0],"%")

accuracy on test set is:  73.24425552153622 %
